In [1]:
import numpy as np
import time
import pandas as pd
from sklearn.datasets import fetch_20newsgroups 

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer 

from sklearn.linear_model import SGDClassifier 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV


## Creating test + train sets

In [2]:
train = pd.read_csv('train_IMDb.csv')
test = pd.read_csv('test_IMDb.csv')

# allData = pd.read_csv('CombinedIMDb.csv')

In [3]:
class IMBD:
    
    def __init__(self,subset = 'train'):
        if (subset == 'train'):
            self.allData = train.to_numpy()
        else:
            self.allData = test.to_numpy()

        np.random.shuffle(self.allData)

        self.data = self.allData[:,0]
        self.target = self.allData[:,1]
        self.target=self.target.astype('int')
      

In [4]:
# categories = ['alt.atheism', 'soc.religion.christian', 
#               'comp.graphics', 'sci.med']

# currently looking at all categories 
# twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
# twenty_test = fetch_20newsgroups(subset='test', shuffle=True)

imbd = IMBD()

In [5]:
model_names = ['SVM', 'Logistic Regression', 'AdaBoost', 'Decision Tree', 'Random Forest']

# just need to add randomforest 
models = [
    LinearSVC(random_state=0,max_iter=2000),
    LogisticRegression(random_state=0,max_iter=1000),
    AdaBoostClassifier(n_estimators=50, learning_rate=1,random_state=0),
    DecisionTreeClassifier(random_state=0),
    RandomForestClassifier(max_depth=2, random_state=0)
]


parameters = [
    {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'tfidf__use_idf': (True, False),
        'clf__loss': ('hinge', 'squared_hinge')
    },
    {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'tfidf__use_idf': (True, False),
        'clf__penalty': ('l2', 'l1')
    },
    {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'tfidf__use_idf': (True, False),
        'clf__learning_rate': (1, 2, 0.5),
        'clf__n_estimators': (10, 50, 100)
    },
    {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'tfidf__use_idf': (True, False),
        'clf__criterion': ('gini', 'entropy'),
    },
    {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'tfidf__use_idf': (True, False),
        'clf__criterion': ('gini', 'entropy'),
    }    
]

In [6]:
def runGridSearchCV(parameter_grids, models, model_names):
    start = time.time()
    i = 0
    best_scores = []
    best_params = []
    for model in models: 
        print("Currently training model: ", model_names[i])

        text_clf = Pipeline([('vect', CountVectorizer()),
                            ('tfidf', TfidfTransformer()),
                            ('clf', model)])

        _  = text_clf.fit(imbd.data, imbd.target)   

        gs_clf = GridSearchCV(text_clf, parameter_grids[i], n_jobs=-1, cv=3,error_score=0.0)
        gs_clf = gs_clf.fit(imbd.data, imbd.target)

        best_scores.append(gs_clf.best_score_)
        best_params.append(gs_clf.best_params_)

        print("Time taken: ", time.time()-start)
        print("Best score : ", gs_clf.best_score_)
        print("Best params: ", gs_clf.best_params_)
        print("\n\n")

        i+=1


In [7]:
def runRandomizedSearchCV(parameter_grids, models, model_names, num_iters=100):
    start = time.time()
    i = 0
    best_scores = []
    best_params = []
    for model in models: 
        print("Currently training model: ", model_names[i])

        text_clf = Pipeline([('vect', CountVectorizer()),
                            ('tfidf', TfidfTransformer()),
                            ('clf', model)])

        _  = text_clf.fit(imbd.data, imbd.target)    

        rs_clf = RandomizedSearchCV(text_clf, parameter_grids[i], n_jobs=1, cv=3,error_score=0.0,n_iter = num_iters, verbose = 2, random_state=0)
        rs_clf = rs_clf.fit(imbd.data, imbd.target)

        best_scores.append(rs_clf.best_score_)
        best_params.append(rs_clf.best_params_)

        print("Time taken: ", time.time()-start)
        print("Best score : ", rs_clf.best_score_)
        print("Best params: ", rs_clf.best_params_)
        print("\n\n")

        i+=1

## Finding Hyperparameters For Each Model

### Linear SVC Parameters

In [81]:
linear_svc_rs = [
    {
        'vect__ngram_range': [(1,1),(1,2)],
        'vect__stop_words': ['english',None],
        'tfidf__use_idf': [True,False],
        'clf__C': [0.01, 0.05, 0.1, 0.5, 1.0, 10, 20 , 30, 40, 50, 60, 70, 80, 90, 100],
        'clf__penalty': ['l2','l1'],
        'clf__loss': ['hinge','squared_hinge'],
    }
]

In [82]:
# Narrowed from randomized search
linear_svc_gs = [
    {
        'vect__ngram_range': [(1,1),(1,2)],
        'vect__stop_words': ['english',None],
        
        'tfidf__use_idf': [True,False],
        'clf__C': [0.01, 1.0, 100.],
        'clf__penalty': ['l2','l1'],
        'clf__loss': ['hinge','squared_hinge'],
    }
]

In [83]:
runRandomizedSearchCV(linear_svc_rs, [models[0]], [model_names[0]])

Currently training model:  SVM
Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] vect__stop_words=None, vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__penalty=l1, clf__loss=hinge, clf__C=0.01 


/home/harsh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError("Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True")
  "Details: \n%r" % (error_score, e), FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.8s remaining:    0.0s


[CV]  vect__stop_words=None, vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__penalty=l1, clf__loss=hinge, clf__C=0.01, total=   8.8s
[CV] vect__stop_words=None, vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__penalty=l1, clf__loss=hinge, clf__C=0.01 


/home/harsh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError("Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True")
  "Details: \n%r" % (error_score, e), FitFailedWarning)


[CV]  vect__stop_words=None, vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__penalty=l1, clf__loss=hinge, clf__C=0.01, total=   9.0s
[CV] vect__stop_words=None, vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__penalty=l1, clf__loss=hinge, clf__C=0.01 


...

[CV]  vect__stop_words=english, vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__penalty=l1, clf__loss=hinge, clf__C=20, total=   2.0s
[CV] vect__stop_words=english, vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__penalty=l1, clf__loss=hinge, clf__C=20 


/home/harsh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError("Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True")
  "Details: \n%r" % (error_score, e), FitFailedWarning)


[CV]  vect__stop_words=english, vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__penalty=l1, clf__loss=hinge, clf__C=20, total=   2.0s
[CV] vect__stop_words=english, vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__penalty=l1, clf__loss=hinge, clf__C=20 


/home/harsh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError("Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True")
  "Details: \n%r" % (error_score, e), FitFailedWarning)
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 58.1min finished


[CV]  vect__stop_words=english, vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__penalty=l1, clf__loss=hinge, clf__C=20, total=   2.0s
Time taken:  3514.392676115036
Best score :  0.90684
Best params:  {'vect__stop_words': None, 'vect__ngram_range': (1, 2), 'tfidf__use_idf': True, 'clf__penalty': 'l2', 'clf__loss': 'squared_hinge', 'clf__C': 10}





In [ ]:
runGridSearchCV(linear_svc_pg, [models[0]], [model_names[0]])

### Logistic Regression Parameters

In [8]:
logistic_regression_rs = [
        {
            'vect__ngram_range': [(1,1),(1,2)],
            'vect__stop_words': ['english',None],
            'tfidf__use_idf': [True,False],
            'clf__C': [0.01, 0.05, 0.1, 0.5, 1.0, 10, 20 , 30, 40, 50, 60, 70, 80, 90, 100],
            'clf__penalty': ['l2','l1','elasticnet'],
            'clf__solver': ['newton-cg','sag','lbfgs','liblinear','saga'],
        }
]

In [10]:
runRandomizedSearchCV(logistic_regression_rs, [models[1]], [model_names[1]])

Currently training model:  Logistic Regression


C:\Users\heath\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 18.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 78.1min finished


Time taken:  4746.452325344086
Best score :  0.90212
Best params:  {'vect__stop_words': None, 'vect__ngram_range': (1, 2), 'tfidf__use_idf': True, 'clf__solver': 'sag', 'clf__penalty': 'l2', 'clf__C': 60}





In [11]:
logistic_regression_pg = [
        {
            'vect__ngram_range': [(1,1)],
            'vect__stop_words': ['english'],
            'tfidf__use_idf': [True],
            'clf__C': [55, 60, 65],
            'clf__penalty': ['l2','l1','elasticnet'],
            'clf__solver': ['newton-cg','sag','lbfgs','liblinear','saga'],
        }
]

In [12]:
runGridSearchCV(logistic_regression_pg, [models[1]], [model_names[1]])

Currently training model:  Logistic Regression


C:\Users\heath\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Time taken:  1224.9758758544922
Best score :  0.8828
Best params:  {'clf__C': 55, 'clf__penalty': 'l2', 'clf__solver': 'saga', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1), 'vect__stop_words': 'english'}





### Random Forest Parameters

In [8]:
random_forest_rs = [
        {
            'vect__ngram_range': [(1,1),(1,2)],
            'vect__stop_words': ['english',None],
            'tfidf__use_idf': [True,False],
            'clf__bootstrap': [True,False],
            'clf__max_depth' : [10,20,30,40,50,60,70,80,90,100,None],
            'clf__n_estimators' : [100, 200, 400, 700, 800, 1000, 1200, 1400, 1600, 1800, 2000],
#             'clf__C': [0.01, 0.05, 0.1, 0.5, 1.0, 10, 20 , 30, 40, 50, 60, 70, 80, 90, 100],
            'clf__min_samples_split': [2,5,10],
            'clf__max_features': ['auto','sqrt'],
#             'clf__penalty': ['l2','l1','elasticnet'],
#             'clf__solver': ['newton-cg','sag','lbfgs','liblinear','saga'],
        }
]

In [ ]:
runRandomizedSearchCV(random_forest_rs, [models[4]], [model_names[4]])

Currently training model:  Random Forest


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Fitting 3 folds for each of 100 candidates, totalling 300 fits


### Decision Tree Parameters

In [13]:
decision_tree_rs = [
        {
            'vect__ngram_range': [(1,1)],
            'vect__stop_words': ['english'],
            'tfidf__use_idf': [True],
            'clf__max_features': [None,'auto','sqrt','log2'],
            'clf__max_depth': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
            'clf__min_impurity_decrease': [0.00005,0.0001,0.0002,0.0005,0.001,0.0015,0.002,0.005,0.01]
        }
]

In [14]:
runRandomizedSearchCV(decision_tree_rs, [models[3]], [model_names[3]])

Currently training model:  Decision Tree
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   43.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.5min finished


Time taken:  491.74345421791077
Best score :  0.72184
Best params:  {'vect__stop_words': 'english', 'vect__ngram_range': (1, 1), 'tfidf__use_idf': True, 'clf__min_impurity_decrease': 0.002, 'clf__max_features': None, 'clf__max_depth': 14}





In [15]:
decision_tree_pg = [
        {
            'vect__ngram_range': [(1,1)],
            'vect__stop_words': ['english'],
            'tfidf__use_idf': [True],
            'clf__max_features': [None,'auto','sqrt','log2'],
            'clf__max_depth': [12,13,14,15,16],
            'clf__min_impurity_decrease': [0.001,0.002,0.003]
        }
]

In [16]:
runGridSearchCV(decision_tree_pg, [models[3]], [model_names[3]])

Currently training model:  Decision Tree
Time taken:  328.3461666107178
Best score :  0.7306
Best params:  {'clf__max_depth': 16, 'clf__max_features': None, 'clf__min_impurity_decrease': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1), 'vect__stop_words': 'english'}





In [17]:
decision_tree_pg2 = [
        {
            'vect__ngram_range': [(1,1)],
            'vect__stop_words': ['english'],
            'tfidf__use_idf': [True],
            'clf__max_features': [None],
            'clf__max_depth': [14,15,16,17],
            'clf__min_impurity_decrease': [0.0005, 0.001,0.002]
        }
]

In [18]:
runGridSearchCV(decision_tree_pg2, [models[3]], [model_names[3]])

Currently training model:  Decision Tree
Time taken:  107.41896724700928
Best score :  0.73568
Best params:  {'clf__max_depth': 17, 'clf__max_features': None, 'clf__min_impurity_decrease': 0.0005, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1), 'vect__stop_words': 'english'}





In [19]:
decision_tree_pg3 = [
        {
            'vect__ngram_range': [(1,1)],
            'vect__stop_words': ['english'],
            'tfidf__use_idf': [True],
            'clf__max_features': [None],
            'clf__max_depth': [16,17,18,19],
            'clf__min_impurity_decrease': [0.0001, 0.0005, 0.001,]
        }
]

In [21]:
runGridSearchCV(decision_tree_pg3, [models[3]], [model_names[3]])

Currently training model:  Decision Tree
Time taken:  126.65679788589478
Best score :  0.73888
Best params:  {'clf__max_depth': 19, 'clf__max_features': None, 'clf__min_impurity_decrease': 0.0005, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1), 'vect__stop_words': 'english'}





In [29]:
decision_tree_pg4 = [
        {
            'vect__ngram_range': [(1,1)],
            'vect__stop_words': ['english'],
            'tfidf__use_idf': [True],
            'clf__max_features': [None],
            'clf__max_depth': [35,40,45],
            'clf__min_impurity_decrease': [0.0005]
        }
]

In [30]:
runGridSearchCV(decision_tree_pg4, [models[3]], [model_names[3]])

Currently training model:  Decision Tree
Time taken:  68.89858627319336
Best score :  0.74916
Best params:  {'clf__max_depth': 40, 'clf__max_features': None, 'clf__min_impurity_decrease': 0.0005, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1), 'vect__stop_words': 'english'}



